In [4]:
import sys,os
folder_name="march_madness"
BASE_DIR=os.path.abspath(".").split(folder_name)[0]+folder_name
DATA_ROOT=os.path.join(BASE_DIR,"data")
#todo setup shared utilities folder
sys.path.insert(0, DATA_ROOT)

In [22]:
import pandas as pd
pd.options.display.max_columns=999
pd.options.display.max_rows=5
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from copy import deepcopy

In [6]:
regular_season_results_df=pd.read_csv(DATA_ROOT+"/DataFiles/RegularSeasonDetailedResults.csv")
regular_season_results_df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,8,20,10,19,15,28,16,13,4,4,18,24,67,6,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,8,18,17,29,17,26,15,10,5,2,25,22,73,3,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,3,9,17,31,6,19,11,12,14,2,18,18,49,6,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,6,14,11,13,17,22,12,14,4,4,20,24,62,6,16,17,27,21,15,12,10,7,1,14


In [102]:
tourney_seeds=pd.read_csv(DATA_ROOT+"/DataFiles/NCAATourneySeeds.csv")
tourney_seeds.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [103]:
import re
def unpack_seed(seed):
    d={}
    d['region']=seed[0]
    d['seed']=int(re.findall("[0-9]+",seed)[0])
    #below doesn't identify 65 vs 67 but seeds past 64 will be ignored in Kaggle comp
    if 'a' in seed:
        d['seed']+=1
    if 'b' in seed:
        d['seed']+=2
    return pd.Series(d)
tourney_seeds=tourney_seeds.join(tourney_seeds['Seed'].apply(unpack_seed))

In [104]:
tourney_seeds

,Season,Seed,TeamID,region,seed
0,1985,W01,1207,W,1
1,1985,W02,1210,W,2
...,...,...,...,...,...
2216,2018,Z16a,1300,Z,17
2217,2018,Z16b,1411,Z,18


In [105]:
tourney_results=pd.read_csv(DATA_ROOT+"/DataFiles/NCAATourneyCompactResults.csv")
for col in tourney_results.columns:
    if "ID" in col:
        tourney_results[col]=tourney_results[col].astype(str)
tourney_results

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
...,...,...,...,...,...,...,...,...
2182,2018,152,1437,95,1242,79,N,0
2183,2018,154,1437,79,1276,62,N,0


In [106]:
teams=pd.read_csv(DATA_ROOT+"/DataFiles/Teams.csv")
team_conferences=pd.read_csv(DATA_ROOT+"/DataFiles/TeamConferences.csv")#contains season

In [107]:
seasons=pd.read_csv(DATA_ROOT+"/DataFiles/Seasons.csv").set_index("Season",drop=False)
seasons['DayZero']=pd.to_datetime(seasons['DayZero'])

In [108]:
import featuretools as ft

In [109]:
#using https://towardsdatascience.com/automated-feature-engineering-for-predictive-modeling-d8c9fa4e478b as a starting point
#interface looks a little old so I am using the current FT docs too
from sklearn.linear_model import LogisticRegression

In [110]:
from random import shuffle
tourney_games=tourney_results.copy()
def get_team1_team2(row):
    d={}
    teams=[row['WTeamID'],row['LTeamID']]
    shuffle(teams)
    d['Team1'],d['Team2']=teams
    return pd.Series(d)
tourney_games=tourney_games.join(tourney_games.apply(get_team1_team2,axis=1))
tourney_games['Team1_wins']=tourney_games['WTeamID']==tourney_games['Team1']
tourney_games=tourney_games[['Season', 'DayNum','Team1', 'Team2','Team1_wins']]
tourney_games.head()

,Season,DayNum,Team1,Team2,Team1_wins
0,1985,136,1116,1234,True
1,1985,136,1120,1345,True
2,1985,136,1207,1250,True
3,1985,136,1229,1425,True
4,1985,136,1242,1325,True


In [111]:
tourney_games

,Season,DayNum,Team1,Team2,Team1_wins
0,1985,136,1116,1234,True
1,1985,136,1120,1345,True
...,...,...,...,...,...
2182,2018,152,1242,1437,False
2183,2018,154,1276,1437,False


In [112]:
def get_game_index(row):
    order=sorted([row['Team1'],row['Team2']])
    return "%s_%s_%s"%(row['Season'],order[0],order[1])
import datetime
def get_game_daytime(row):
    #not an acutal "day". This uses Season and DayNum without a first day to index the day_num
    #use seasons to get dayzero
    game_day=seasons.loc[row['Season']]['DayZero']+datetime.timedelta(days=row['DayNum'])
    return game_day
def get_team_season_id(row,team_accessor="TeamID"):
    return "%s_%s"%(row['Season'],row[team_accessor])
tourney_games['game_index']=tourney_games.apply(get_game_index,axis=1)
tourney_games['team1_season_id']=tourney_games.apply(get_team_season_id,args=("Team1",),axis=1)
tourney_games['team2_season_id']=tourney_games.apply(get_team_season_id,args=("Team2",),axis=1)
tourney_games['game_datetime']=tourney_games.apply(get_game_daytime, axis=1)
tourney_seeds['team_season_id']=tourney_seeds.apply(get_team_season_id,axis=1)
tourney_seeds['time_index']=tourney_seeds['Season'].apply(lambda x: datetime.datetime(x, 1,1))


In [113]:
tourney_seeds_full=tourney_seeds.merge(seasons.reset_index(drop=True))
def getRegion(row):
    return row['Region'+row['region']]
tourney_seeds_full['determined_region']=tourney_seeds_full.apply(getRegion,axis=1)
tourney_seeds_full=tourney_seeds_full[['Season', 'TeamID', 'region', 'seed', 'team_season_id',
       'time_index',
       'determined_region']]
tourney_seeds_full.head()

,Season,TeamID,region,seed,team_season_id,time_index,determined_region
0,1985,1207,W,1,1985_1207,1985-01-01,East
1,1985,1210,W,2,1985_1210,1985-01-01,East
2,1985,1228,W,3,1985_1228,1985-01-01,East
3,1985,1260,W,4,1985_1260,1985-01-01,East
4,1985,1374,W,5,1985_1374,1985-01-01,East


In [118]:
team_1_game_logs=tourney_games.copy()
team_1_game_logs=team_1_game_logs.rename(columns={"Team1": "TeamID", "Team1_wins":"wins"})
del team_1_game_logs['Team2']
team_2_game_logs=tourney_games.copy()
team_2_game_logs=team_2_game_logs.rename(columns={"Team2": "TeamID", "Team1_wins":"wins"})
team_2_game_logs['wins']=~team_2_game_logs['wins']
del team_2_game_logs['Team1']
tourney_game_logs=pd.concat([team_1_game_logs,team_2_game_logs],sort=True)
tourney_game_logs=tourney_game_logs.sort_values(by=["game_datetime","game_index"])
tourney_game_logs['team_season_id']=tourney_game_logs.apply(get_team_season_id,axis=1)
tourney_game_logs['game_log_index']=tourney_game_logs['game_index']+tourney_game_logs['TeamID']

In [119]:
tourney_game_logs

,DayNum,Season,TeamID,game_datetime,game_index,team1_season_id,team2_season_id,wins,team_season_id,game_log_index
0,136,1985,1116,1985-03-14,1985_1116_1234,1985_1116,1985_1234,True,1985_1116,1985_1116_12341116
0,136,1985,1234,1985-03-14,1985_1116_1234,1985_1116,1985_1234,False,1985_1234,1985_1116_12341234
...,...,...,...,...,...,...,...,...,...,...
2183,154,2018,1276,2018-04-02,2018_1276_1437,2018_1276,2018_1437,False,2018_1276,2018_1276_14371276
2183,154,2018,1437,2018-04-02,2018_1276_1437,2018_1276,2018_1437,True,2018_1437,2018_1276_14371437


In [399]:
regular_season_results_df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,game_index,game_datetime
0,2003,10,1104,68,1328,62,N,0,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20,200310_11041328,2003-01-11
1,2003,10,1272,70,1393,63,N,0,26,62,8,20,10,19,15,28,16,13,4,4,18,24,67,6,24,9,20,20,25,7,12,8,6,16,200310_12721393,2003-01-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82039,2018,132,1246,77,1397,72,N,0,25,50,7,16,20,24,11,21,9,10,6,6,19,23,62,10,27,16,20,18,16,14,10,4,3,21,2018132_12461397,2018-05-13
82040,2018,132,1335,68,1217,65,N,0,24,52,8,18,12,16,3,27,10,7,3,3,14,23,55,7,18,12,12,5,29,6,8,2,2,17,2018132_13351217,2018-05-13


In [400]:
import datetime
def get_game_index(row):
    return "%s%s_%s%s"%(row['Season'],row['DayNum'],row['WTeamID'],row['LTeamID'])

def get_game_daytime(row):
    #not an acutal "day". This uses Season and DayNum without a first day to index the day_num
    
    game_day=seasons.loc[row['Season']]['DayZero']+datetime.timedelta(days=row['DayNum'])
    return game_day
    
regular_season_results_df['game_index']=regular_season_results_df.apply(get_game_index, axis=1)
regular_season_results_df['game_datetime']=pd.to_datetime(regular_season_results_df.apply(get_game_daytime, axis=1))
game=regular_season_results_df.iloc[0]
def get_team_game_log(results_df,winner=True):
    team_game_log=regular_season_results_df.copy()
    prefixes=["team_","opp_"]
    if not winner:
        prefixes=["opp_","team_"]
    rename_dict={}
    for col in regular_season_results_df.columns:
        if col=="WLoc":
            continue
        elif col.startswith("W"):
            rename_dict[col]=prefixes[0]+col[1:]
        elif col.startswith("L") and col not in ["Loc"]:
            rename_dict[col]=prefixes[1]+col[1:]
    team_game_log=team_game_log.rename(columns=rename_dict)
    return team_game_log
winner_games=get_team_game_log(regular_season_results_df, winner=True)
loser_games=get_team_game_log(regular_season_results_df, winner=False)
team_game_log=pd.concat([winner_games,loser_games],sort=False)
team_game_log=team_game_log.sort_values(by=["Season","DayNum","game_index"])
team_game_log['team_margin']=team_game_log['team_Score']-team_game_log['opp_Score']
team_game_log['team_won']=team_game_log['team_margin']>0
#teams can't tie in basketball, above code assumes that
assert(len(team_game_log[team_game_log['team_margin']==0])==0)

In [419]:
team_game_log['team_TeamID']=team_game_log['team_TeamID'].astype(str)
team_game_log['team_season_id']=team_game_log['Season'].astype(str)+"_"+team_game_log['team_TeamID']
team_game_log['team_game_index']=team_game_log['game_index']+team_game_log['team_TeamID']

In [420]:
tourney_game_logs

,DayNum,Season,TeamID,game_datetime,game_index,team1_season_id,team2_season_id,wins,team_season_id,game_log_index
0,136,1985,1116,1985-03-14,1985_1116_1234,1985_1116,1985_1234,True,1985_1116,1985_1116_12341116
0,136,1985,1234,1985-03-14,1985_1116_1234,1985_1116,1985_1234,False,1985_1234,1985_1116_12341234
...,...,...,...,...,...,...,...,...,...,...
2183,154,2018,1276,2018-04-02,2018_1276_1437,2018_1276,2018_1437,False,2018_1276,2018_1276_14371276
2183,154,2018,1437,2018-04-02,2018_1276_1437,2018_1276,2018_1437,True,2018_1437,2018_1276_14371437


In [421]:

base_es = ft.EntitySet()
#compound key index?
base_es = base_es.entity_from_dataframe(entity_id="tourney_game_logs",
                             dataframe=tourney_game_logs,
                              index="game_log_index",
                              time_index="game_datetime"
                             )
base_es = base_es.entity_from_dataframe(entity_id="regular_game_logs",
                             dataframe=team_game_log,
                              index="team_game_index",
                              time_index="game_datetime"
                             )

base_es = base_es.entity_from_dataframe(entity_id="tourney_seeds",
                             dataframe=tourney_seeds_full,
                              index="team_season_id",
                              time_index="time_index"
                             )

# es = es.add_relationship(
#     ft.Relationship(
#                     es['seasons']['Season'],
        
#                     es['tourney_seeds']['Season'], 
#                    )
# )

# es = es.add_relationship(
#     ft.Relationship(
#                     es['tourney_seeds']['team_season_id'],
        
#                     es['tourney_games']['team2_season_id'], 
#                    )
# )
#es.add_last_time_indexes()

In [422]:
#"tourney_games": ["Team1","Team2","DayNum","Season", "team1_season_id","team2_season_id","Team1_wins"],
                                              

In [423]:
es=deepcopy(base_es)
es = es.add_relationship(
    ft.Relationship(
                    es['tourney_seeds']['team_season_id'],

                    es['tourney_game_logs']["team_season_id"], 
                   )
)
es = es.add_relationship(
    ft.Relationship(
                    es['regular_game_logs']['team_season_id'],

                    es['tourney_game_logs']["team_season_id"], 
                   )
)
es.relationships
feature_matrix, feature_defs = ft.dfs(entityset=es,
                                      target_entity="tourney_game_logs",
                                      verbose=True,
#                                       cutoff_time=datetime.datetime(2016,5,20),
#                                       cutoff_time_in_index=True,
                                      ignore_variables={
                                          "tourney_game_logs": ["DayNum","Season", "team1_season_id","team2_season_id","game_datetime"],
                                          "tourney_seeds": ["Seed","DayNum","Season", "TeamID", "team_game_index","time_index"]
                                      },
                                      agg_primitives=[],
                                      trans_primitives=["month", "weekday"]

                                     )
feature_matrix.head()

AttributeError: Parent variable '<Variable: team_season_id (dtype = categorical)>' is not the index of entity Entity: regular_game_logs
  Variables:
    team_game_index (dtype: index)
    Season (dtype: numeric)
    DayNum (dtype: numeric)
    team_TeamID (dtype: categorical)
    team_Score (dtype: numeric)
    opp_TeamID (dtype: numeric)
    opp_Score (dtype: numeric)
    WLoc (dtype: categorical)
    NumOT (dtype: numeric)
    team_FGM (dtype: numeric)
    team_FGA (dtype: numeric)
    team_FGM3 (dtype: numeric)
    team_FGA3 (dtype: numeric)
    team_FTM (dtype: numeric)
    team_FTA (dtype: numeric)
    team_OR (dtype: numeric)
    team_DR (dtype: numeric)
    team_Ast (dtype: numeric)
    team_TO (dtype: numeric)
    team_Stl (dtype: numeric)
    team_Blk (dtype: numeric)
    team_PF (dtype: numeric)
    opp_FGM (dtype: numeric)
    opp_FGA (dtype: numeric)
    opp_FGM3 (dtype: numeric)
    opp_FGA3 (dtype: numeric)
    opp_FTM (dtype: numeric)
    opp_FTA (dtype: numeric)
    opp

In [124]:
to_encode=[col for col in feature_matrix if "id" not in col.lower() and "index" not in col.lower() and col!="wins"]
fm_encoded, features_encoded = ft.encode_features(feature_matrix, feature_defs, to_encode=to_encode)

In [125]:
fm_encoded

,TeamID,game_index,wins,team_season_id,tourney_seeds.region = Y,tourney_seeds.region = W,tourney_seeds.region = Z,tourney_seeds.region = X,tourney_seeds.region is unknown,tourney_seeds.seed,tourney_seeds.determined_region = East,tourney_seeds.determined_region = West,tourney_seeds.determined_region = Midwest,tourney_seeds.determined_region = South,tourney_seeds.determined_region = Southeast,tourney_seeds.determined_region = Atlanta,tourney_seeds.determined_region = Syracuse,tourney_seeds.determined_region = StLouis,tourney_seeds.determined_region = Minneapolis,tourney_seeds.determined_region = Southwest,tourney_seeds.determined_region is unknown
game_log_index,,,,,,,,,,,,,,,,,,,,,
1985_1104_11121104,1104,1985_1104_1112,True,1985_1104,0,0,0,1,0,7,0,1,0,0,0,0,0,0,0,0,0
1985_1104_11121112,1112,1985_1104_1112,False,1985_1112,0,0,0,1,0,10,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018_1437_14521437,1437,2018_1437_1452,True,2018_1437,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0
2018_1437_14521452,1452,2018_1437_1452,False,2018_1452,0,1,0,0,0,5,1,0,0,0,0,0,0,0,0,0,0


In [358]:
game

,TeamID,game_index,wins,team_season_id,tourney_seeds.region = Y,tourney_seeds.region = W,tourney_seeds.region = Z,tourney_seeds.region = X,tourney_seeds.region is unknown,tourney_seeds.seed,tourney_seeds.determined_region = East,tourney_seeds.determined_region = West,tourney_seeds.determined_region = Midwest,tourney_seeds.determined_region = South,tourney_seeds.determined_region = Southeast,tourney_seeds.determined_region = Atlanta,tourney_seeds.determined_region = Syracuse,tourney_seeds.determined_region = StLouis,tourney_seeds.determined_region = Minneapolis,tourney_seeds.determined_region = Southwest,tourney_seeds.determined_region is unknown
game_log_index,,,,,,,,,,,,,,,,,,,,,
2018_1437_14521437,1437,2018_1437_1452,True,2018_1437,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0
2018_1437_14521452,1452,2018_1437_1452,False,2018_1452,0,1,0,0,0,5,1,0,0,0,0,0,0,0,0,0,0


In [377]:
#could also do a join based on win split but this is more generic for other sports because ties
merged_features=pd.DataFrame()
for game_index, game in fm_encoded.groupby("game_index"):
    assert(len(game)==2)
    merged=pd.DataFrame([game.iloc[0]]).merge(pd.DataFrame([game.iloc[1]]), on="game_index", suffixes=["_team1","_team2"])
    merged2=pd.DataFrame([game.iloc[0]]).merge(pd.DataFrame([game.iloc[1]]), on="game_index", suffixes=["_team2","_team1"])
    merged_features=merged_features.append(merged).append(merged2)
    

C:\Users\MatteoHoch\Anaconda3\envs\march_madness\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [378]:
#merged_features=merged_features[(merged_features['tourney_seeds.seed_team1']<=16)&(merged_features['tourney_seeds.seed_team2']<=16)]

In [379]:
y=merged_features['wins_team1']
keep_cols=[col for col in merged_features if "id" not in col.lower() and "index" not in col.lower() and "wins" not in col]
X=merged_features[keep_cols].fillna(0)
len(y),len(X)

(4368, 4368)

In [380]:
from sklearn.ensemble import *
from sklearn.model_selection import cross_val_score
from sklearn.metrics import *
clf = RandomForestClassifier(n_estimators=400, n_jobs=-1)
scores = cross_val_score(estimator=clf,X=X, y=y, cv=3,
                         scoring="neg_log_loss", verbose=True)

"AUC %.2f +/- %.2f" % (scores.mean(), scores.std())

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    2.1s finished


'AUC -0.97 +/- 0.08'

In [383]:
X

,tourney_seeds.determined_region = Atlanta_team1,tourney_seeds.determined_region = Atlanta_team2,tourney_seeds.determined_region = East_team1,tourney_seeds.determined_region = East_team2,tourney_seeds.determined_region = Minneapolis_team1,tourney_seeds.determined_region = Minneapolis_team2,tourney_seeds.determined_region = South_team1,tourney_seeds.determined_region = South_team2,tourney_seeds.determined_region = Southeast_team1,tourney_seeds.determined_region = Southeast_team2,tourney_seeds.determined_region = Southwest_team1,tourney_seeds.determined_region = Southwest_team2,tourney_seeds.determined_region = StLouis_team1,tourney_seeds.determined_region = StLouis_team2,tourney_seeds.determined_region = Syracuse_team1,tourney_seeds.determined_region = Syracuse_team2,tourney_seeds.determined_region = West_team1,tourney_seeds.determined_region = West_team2,tourney_seeds.determined_region is unknown_team1,tourney_seeds.determined_region is unknown_team2,tourney_seeds.region = W_team1,tourney_seeds.region = W_team2,tourney_seeds.region = X_team1,tourney_seeds.region = X_team2,tourney_seeds.region = Y_team1,tourney_seeds.region = Y_team2,tourney_seeds.region = Z_team1,tourney_seeds.region = Z_team2,tourney_seeds.region is unknown_team1,tourney_seeds.region is unknown_team2,tourney_seeds.seed_team1,tourney_seeds.seed_team2
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,7,10
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,10,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,5
0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,5,1


In [384]:

clf.fit(X, y)



array([0.00116924, 0.00122452, 0.00266127, 0.00276918, 0.00180876,
       0.00174134, 0.00718519, 0.00724634, 0.00791768, 0.00750667,
       0.00201728, 0.00217864, 0.00149434, 0.00152675, 0.00141044,
       0.0015592 , 0.00562554, 0.00612404, 0.00505   , 0.00498577,
       0.00340192, 0.0034714 , 0.00587844, 0.0063769 , 0.00573662,
       0.00569089, 0.00565306, 0.00584761, 0.        , 0.        ,
       0.44327839, 0.44146257])

In [390]:
type(X.columns[0])==str

True

In [392]:
import utils
from importlib import reload
reload(utils)
utils.feature_importances(clf, X.columns, n=300)

1: Feature: tourney_seeds.seed_team1, 0.443
2: Feature: tourney_seeds.seed_team2, 0.441
3: Feature: tourney_seeds.determined_region = Southeast_team1, 0.008
4: Feature: tourney_seeds.determined_region = Southeast_team2, 0.008
5: Feature: tourney_seeds.determined_region = South_team2, 0.007
6: Feature: tourney_seeds.determined_region = South_team1, 0.007
7: Feature: tourney_seeds.region = X_team2, 0.006
8: Feature: tourney_seeds.determined_region = West_team2, 0.006
9: Feature: tourney_seeds.region = X_team1, 0.006
10: Feature: tourney_seeds.region = Z_team2, 0.006
11: Feature: tourney_seeds.region = Y_team1, 0.006
12: Feature: tourney_seeds.region = Y_team2, 0.006
13: Feature: tourney_seeds.region = Z_team1, 0.006
14: Feature: tourney_seeds.determined_region = West_team1, 0.006
15: Feature: tourney_seeds.determined_region is unknown_team1, 0.005
16: Feature: tourney_seeds.determined_region is unknown_team2, 0.005
17: Feature: tourney_seeds.region = W_team2, 0.003
18: Feature: tourney_s

['tourney_seeds.seed_team1',
 'tourney_seeds.seed_team2',
 'tourney_seeds.determined_region = Southeast_team1',
 'tourney_seeds.determined_region = Southeast_team2',
 'tourney_seeds.determined_region = South_team2',
 'tourney_seeds.determined_region = South_team1',
 'tourney_seeds.region = X_team2',
 'tourney_seeds.determined_region = West_team2',
 'tourney_seeds.region = X_team1',
 'tourney_seeds.region = Z_team2',
 'tourney_seeds.region = Y_team1',
 'tourney_seeds.region = Y_team2',
 'tourney_seeds.region = Z_team1',
 'tourney_seeds.determined_region = West_team1',
 'tourney_seeds.determined_region is unknown_team1',
 'tourney_seeds.determined_region is unknown_team2',
 'tourney_seeds.region = W_team2',
 'tourney_seeds.region = W_team1',
 'tourney_seeds.determined_region = East_team2',
 'tourney_seeds.determined_region = East_team1',
 'tourney_seeds.determined_region = Southwest_team2',
 'tourney_seeds.determined_region = Southwest_team1',
 'tourney_seeds.determined_region = Minneapo

In [395]:
X

,tourney_seeds.determined_region = Atlanta_team1,tourney_seeds.determined_region = Atlanta_team2,tourney_seeds.determined_region = East_team1,tourney_seeds.determined_region = East_team2,tourney_seeds.determined_region = Minneapolis_team1,tourney_seeds.determined_region = Minneapolis_team2,tourney_seeds.determined_region = South_team1,tourney_seeds.determined_region = South_team2,tourney_seeds.determined_region = Southeast_team1,tourney_seeds.determined_region = Southeast_team2,tourney_seeds.determined_region = Southwest_team1,tourney_seeds.determined_region = Southwest_team2,tourney_seeds.determined_region = StLouis_team1,tourney_seeds.determined_region = StLouis_team2,tourney_seeds.determined_region = Syracuse_team1,tourney_seeds.determined_region = Syracuse_team2,tourney_seeds.determined_region = West_team1,tourney_seeds.determined_region = West_team2,tourney_seeds.determined_region is unknown_team1,tourney_seeds.determined_region is unknown_team2,tourney_seeds.region = W_team1,tourney_seeds.region = W_team2,tourney_seeds.region = X_team1,tourney_seeds.region = X_team2,tourney_seeds.region = Y_team1,tourney_seeds.region = Y_team2,tourney_seeds.region = Z_team1,tourney_seeds.region = Z_team2,tourney_seeds.region is unknown_team1,tourney_seeds.region is unknown_team2,tourney_seeds.seed_team1,tourney_seeds.seed_team2
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,7,10
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,10,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,5
0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,5,1


In [393]:
predictions=fm_encoded.copy()
#TODO remove play in games by seeds 16a and 16b
np=pd.np
predictions['y_hat']=np.clip(clf.predict_proba(X)[:,1],.3,.7)
#predictions=predictions.sort_values(by=["game_index","TeamID"]).drop_duplicates(subset=["game_index"],keep="first")
predictions['Season']=predictions['game_index'].apply(lambda x: x.split("_")[0]).astype(float)

log_loss(predictions[predictions['Season']>=2014]['wins'],predictions[predictions['Season']>=2014]['y_hat'])

0.5251792065293455

In [310]:
predictions

,TeamID,game_index,wins,team_season_id,tourney_seeds.region = Y,tourney_seeds.region = W,tourney_seeds.region = Z,tourney_seeds.region = X,tourney_seeds.region is unknown,tourney_seeds.seed,tourney_seeds.determined_region = East,tourney_seeds.determined_region = West,tourney_seeds.determined_region = Midwest,tourney_seeds.determined_region = South,tourney_seeds.determined_region = Southeast,tourney_seeds.determined_region = Atlanta,tourney_seeds.determined_region = Syracuse,tourney_seeds.determined_region = StLouis,tourney_seeds.determined_region = Minneapolis,tourney_seeds.determined_region = Southwest,tourney_seeds.determined_region is unknown,y_hat,Season
game_log_index,,,,,,,,,,,,,,,,,,,,,,,
1985_1104_11121104,1104,1985_1104_1112,True,1985_1104,0,0,0,1,0,7,0,1,0,0,0,0,0,0,0,0,0,0.7,1985.0
1985_1104_13011104,1104,1985_1104_1301,False,1985_1104,0,0,0,1,0,7,0,1,0,0,0,0,0,0,0,0,0,0.3,1985.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018_1420_14381420,1420,2018_1420_1438,True,2018_1420,1,0,0,0,0,16,0,0,0,1,0,0,0,0,0,0,0,0.3,2018.0
2018_1437_14521437,1437,2018_1437_1452,True,2018_1437,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0.7,2018.0


In [290]:
from itertools import combinations
tourney_seeds=pd.read_csv(DATA_ROOT+"/DataFiles/NCAATourneySeeds.csv")
possible_games_df=pd.DataFrame()
for season, seeds in tourney_seeds.groupby("Season"):
    possible_games=list(combinations(tourney_seeds[tourney_seeds['Season']==season]['TeamID'],2))
    df=pd.DataFrame(possible_games)
    df['Season']=season
    possible_games_df=possible_games_df.append(df)
possible_games_df=possible_games_df[possible_games_df['Season']>=2014]
possible_games_df['game_index']=possible_games_df.apply(lambda row: "%s_%s_%s"%(row['Season'],min([row[0],row[1]]),max([row[0],row[1]])), axis=1)

possible_games_df['TeamID']=possible_games_df.apply(lambda row: min([row[0],row[1]]),axis=1)
possible_games_df['ID']=possible_games_df.apply(lambda row: "%s_%s_%s"%(row['Season'],min([row[0],row[1]]),max([row[0],row[1]])), axis=1)

In [291]:
#predictions=predictions[predictions.apply(lambda row: row['team_season_id'] in row['game_index'], axis=1)]

In [292]:
predictions.groupby("Season").count()

,TeamID,game_index,wins,team_season_id,tourney_seeds.region = Y,tourney_seeds.region = W,tourney_seeds.region = Z,tourney_seeds.region = X,tourney_seeds.region is unknown,tourney_seeds.seed,tourney_seeds.determined_region = East,tourney_seeds.determined_region = West,tourney_seeds.determined_region = Midwest,tourney_seeds.determined_region = South,tourney_seeds.determined_region = Southeast,tourney_seeds.determined_region = Atlanta,tourney_seeds.determined_region = Syracuse,tourney_seeds.determined_region = StLouis,tourney_seeds.determined_region = Minneapolis,tourney_seeds.determined_region = Southwest,tourney_seeds.determined_region is unknown,y_hat
Season,,,,,,,,,,,,,,,,,,,,,,
1985,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63
1986,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67
2018,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67


In [293]:
submission=possible_games_df.merge(predictions, on=["game_index","TeamID"],how="left",suffixes=["","_2"])
submission[~pd.isnull(submission['y_hat'])].groupby("Season").count()

,0,1,game_index,TeamID,ID,wins,team_season_id,tourney_seeds.region = Y,tourney_seeds.region = W,tourney_seeds.region = Z,tourney_seeds.region = X,tourney_seeds.region is unknown,tourney_seeds.seed,tourney_seeds.determined_region = East,tourney_seeds.determined_region = West,tourney_seeds.determined_region = Midwest,tourney_seeds.determined_region = South,tourney_seeds.determined_region = Southeast,tourney_seeds.determined_region = Atlanta,tourney_seeds.determined_region = Syracuse,tourney_seeds.determined_region = StLouis,tourney_seeds.determined_region = Minneapolis,tourney_seeds.determined_region = Southwest,tourney_seeds.determined_region is unknown,y_hat,Season_2
Season,,,,,,,,,,,,,,,,,,,,,,,,,,
2003,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64
2004,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67
2018,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67


In [294]:
possible_games_df[possible_games_df['game_index']=='2018_1420_1438']

,0,1,Season,game_index,TeamID,ID
1764,1438,1420,2018,2018_1420_1438,1420,2018_1420_1438


In [296]:
submission['Pred']=submission['y_hat'].astype(float).fillna(.5)

In [297]:
submission['Pred'].unique()

array([0.5       , 0.40765949, 0.3       , 0.7       , 0.34145613,
       0.3860855 , 0.3609217 , 0.50152381, 0.32732576, 0.49475132,
       0.36920918, 0.35389023, 0.45178119, 0.49168552, 0.52109199,
       0.51602058, 0.50484482, 0.37870236, 0.63050625, 0.52340682,
       0.60775189, 0.31818554, 0.32522511, 0.6042463 , 0.57984284,
       0.36212192, 0.41595818, 0.43226286, 0.67543708, 0.56198705,
       0.43256027, 0.56542896, 0.45422884, 0.34412179, 0.52937807,
       0.44194486, 0.67561111, 0.52219859, 0.65033036, 0.34576538,
       0.67290882, 0.34099008, 0.3681967 , 0.51450577, 0.41568199,
       0.56927976, 0.42694689, 0.62098731, 0.62952083, 0.4842619 ,
       0.63333333, 0.68096483, 0.67252488, 0.5304176 , 0.32345328,
       0.64779   , 0.31710119, 0.56785004, 0.5822619 , 0.35633631,
       0.61718515, 0.31047761, 0.50587245, 0.30390146, 0.5640242 ,
       0.64705251, 0.34348075, 0.67892021, 0.42479168, 0.34066071,
       0.60221943, 0.69700806, 0.31903716, 0.6056702 , 0.61703

In [298]:
len(possible_games_df)

34864

In [299]:
len(submission)

34864

In [270]:
submission[['ID','Pred']].to_csv("submission_seeds_model.csv",index=False)

In [197]:
lr = LogisticRegression()
model = lr.fit(X, y)
model.score(X, y)

C:\Users\MatteoHoch\Anaconda3\envs\march_madness\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.6967777984727137

# FT Ideas
- Seasonal data?  Will the "Training window" work? https://docs.featuretools.com/automated_feature_engineering/handling_time.html#training-window-and-the-last-time-index
- Compound key?  
- win record? should I try to prep data so FT will create it and the derived fields like last 10 games? Or code manually?
- oppositional data? In the past I've done home_team, home_opp, away_team,away_opp (team and opp loosely correspond to OFF and DEF).

# Go Through Tuning
https://docs.featuretools.com/guides/tuning_dfs.html  
conferences are "interesting_values"  
go deeper into the primitives

# Feature Ideas
Calculate OFF Rating,  
DEF Rating,  
Net rating.  
https://www.kaggle.com/pozz13/ncaa-2k19-eda-zion-s-kingdom shows net_rating_mean as a good feature.  
  
Model games with noise and rankings systems (seeds) as fuzzy. Middle seeds vary a lot in terms of true ranking.  
Non-Elo approach https://www.kaggle.com/raddar/team-power-rankings   
Basketball Advanced Stat Gloassary https://www.basketball-reference.com/about/glossary.html https://www.kaggle.com/bshyammm/starter-kernel-with-model-tune-using-fastai

In [ ]:

fms=[]
feature_dfs=[]
for index in ["team1_season_id","team2_season_id"]:
    es=deepcopy(base_es)
    es = es.add_relationship(
        ft.Relationship(
                        es['tourney_seeds']['team_season_id'],

                        es['tourney_games'][index], 
                       )
    )
    feature_matrix, feature_defs = ft.dfs(entityset=es,
                                          target_entity="tourney_games",
                                          verbose=True,
    #                                       cutoff_time=datetime.datetime(2016,5,20),
    #                                       cutoff_time_in_index=True,
                                          ignore_variables={
                                              "tourney_games": ["Team1","Team2","DayNum","Season", "team1_season_id","team2_season_id","Team1_wins"],
                                              "tourney_seeds": ["Seed","DayNum","Season", "TeamID", "team_game_index","time_index"]
                                          },
                                          agg_primitives=[],

                                         )

    # encode categorical values
    fm_encoded, features_encoded = ft.encode_features(feature_matrix, feature_defs)

    print("Number of features %s" % len(features_encoded))
    fm_encoded.head(10)
    feature_defs+=features_encoded
    fms.append(fm_encoded)

In [177]:
fm_encoded=fms[0].join(fms[1],lsuffix="_team1",rsuffix="_team2")